In [37]:
import pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import pickle
import zlib

In [38]:
DEBUG = False

# OrthoGroups to MSA 
+ load table with the proteins - orthogrous distribution 
+ filter the core orthogroups 
+ load proteins .pkl database
+ get the proteins and the genome positions corresponding to the core OGs

In [39]:
if DEBUG:
    ids = 'korea2' #'korea2' #'rso_test'  #

    core_ogs = '/Users/devseeva/Desktop/work/rso/OrthoFinder/orthoF_msa_'+ids+'/Species_Tree/Orthogroups_for_concatenated_alignment.txt'
    ogs2prot = '/Users/devseeva/Desktop/work/rso/OrthoFinder/orthoF_msa_'+ids+'/Orthogroups/Orthogroups.tsv'
    prot_db = pd.read_pickle('../'+ids+'_outputs/GENOME_PROTEIN_DB_'+ids+'.pkl')
    path_p_aln = '/Users/devseeva/Desktop/work/rso/OrthoFinder/orthoF_msa_'+ids+'/MultipleSequenceAlignments/'

    msa_out_fasta = '/Users/devseeva/Desktop/work/sm_workflow/snakefiles/'+ids+'_concat.fasta'
    msa_out_phylip = '/Users/devseeva/Desktop/work/sm_workflow/snakefiles/'+ids+'_concat.nexus'
    msa_out_xmfa_path = '/Users/devseeva/Desktop/work/sm_workflow/snakefiles/'+ids+'_concat.xmfa'
    aln2acc = '/Users/devseeva/Desktop/work/sm_workflow/snakefiles/outputs/MSA_ACC_DB_'+ids+'.pkl'
else:
    core_ogs = snakemake.input["core_ogs"]
    ogs2prot = snakemake.input["ogs2prot"]
    prot_db = pd.read_pickle(snakemake.input["prot_db"])
    path_p_aln = snakemake.input["path2aln"]

    msa_out_fasta = snakemake.output["msa_fasta"]
    msa_out_phylip = snakemake.output["msa_phylip"]
    msa_out_xmfa_path = snakemake.output["msa_xmfa"]
    aln2acc = snakemake.output["aln2acc"]

In [40]:
strainsNames2fileNames = {}
for c in prot_db.columns:
    strainsNames2fileNames[c.split('$')[1]] = c.split('$')[0]
strainsNames2fileNames

{'to53': 'to53',
 'pe1': 'pe1',
 'pe15': 'pe15',
 'pe3': 'pe3',
 'pe13': 'pe13',
 'pe39': 'pe39',
 'pe61': 'pe61',
 'to7': 'to7',
 'to36': 'to36',
 'to1': 'to1',
 'pe56': 'pe56',
 'pe57': 'pe57',
 'to28': 'to28',
 'pe9': 'pe9',
 'pe27': 'pe27',
 'pe26': 'pe26',
 'to63': 'to63',
 'pe18': 'pe18',
 'pe30': 'pe30'}

In [41]:
prot_db = prot_db.rename(columns=lambda x: str(x).split('$')[0])
prot_db

,to53,pe1,pe15,pe3,pe13,pe39,pe61,to7,to36,to1,pe56,pe57,to28,pe9,pe27,pe26,to63,pe18,pe30
OG0002849,{'nucl': b'x\x9cE\x94\xd1\x91\xc40\x08Ckc\xf8\...,{'nucl': b'x\x9cE\x94\xd1\x11\xc30\x08Cg\xe3\x...,{'nucl': b'x\x9cE\x94\xd1\x91\xc40\x08Ckc\xf8\...,{'nucl': b'x\x9cE\x94\xd1\x91\xc40\x08Ckc\xf8\...,{'nucl': b'x\x9cE\x94\xd1\x91\xc40\x08Ckc\xf8\...,{'nucl': b'x\x9cE\x94\xd1\x11\xc30\x08Cg\xe3\x...,{'nucl': b'x\x9cE\x94\xd1\x11\xc30\x08Cg\xe3\x...,{'nucl': b'x\x9cE\x94\xd1\x11\xc30\x08Cg\xe3\x...,{'nucl': b'x\x9cE\x94\xd1\x91\xc40\x08Ckc\xf8\...,{'nucl': b'x\x9cE\x94\xd1\x91\xc40\x08Ckc\xf8\...,{'nucl': b'x\x9cE\x94\xd1\x11\xc30\x08Cg\xe3\x...,{'nucl': b'x\x9cE\x94\xdb\x91\xc30\x0c\x03k\xe...,{'nucl': b'x\x9cE\x94\xd1\x91\xc40\x08Ckc\xf8\...,{'nucl': b'x\x9cE\x94\xd1\x91\xc40\x08Ckc\xf8\...,{'nucl': b'x\x9cE\x94\xd1\x11\xc30\x08Cg\xe3\x...,{'nucl': b'x\x9cE\x94\xd1\x11\xc30\x08Cg\xe3\x...,{'nucl': b'x\x9cE\x94\xd1\x11\xc30\x08Cg\xe3\x...,{'nucl': b'x\x9cE\x94\xd1\x91\xc40\x08Ckc\xf8\...,{'nucl': b'x\x9cE\x94\xd1\x91\xc40\x08Ckc\xf8\...
OG0001266,{'nucl': b'x\x9cE\x93\xd1\x11\xc30\x08Cg\xd3\x...,{'nucl': b'x\x9cM\x93\xd1\r\x031\x08Cg\xb3\xf8...,{'nucl': b'x\x9cE\x93\xd1\r\xc3@\x08Cg\xb3\xf8...,{'nucl': b'x\x9cE\x93\xd1\r\xc3@\x08Cg\xb3\xf8...,{'nucl': b'x\x9cE\x93\xd1\r\xc3@\x08Cg\xb3\xf8...,{'nucl': b'x\x9cM\x93\xd1\r\x031\x08Cg\xb3\xf8...,{'nucl': b'x\x9cM\x93\xd1\r\x031\x08Cg\xb3\xf8...,{'nucl': b'x\x9cM\x93\xd1\r\x031\x08Cg\xb3\xf8...,{'nucl': b'x\x9cE\x93\xd1\r\xc3@\x08Cg\xb3\xf8...,{'nucl': b'x\x9cE\x93\xd1\r\xc3@\x08Cg\xb3\xf8...,{'nucl': b'x\x9cM\x93\xd1\r\x031\x08Cg\xb3\xf8...,{'nucl': b'x\x9cES\xdb\x11\xc30\x08\x9bM\xa7\x...,{'nucl': b'x\x9cE\x93\xd1\r\xc3@\x08Cg\xb3\xf8...,{'nucl': b'x\x9cE\x93\xd1\r\xc3@\x08Cg\xb3\xf8...,{'nucl': b'x\x9cM\x93\xd1\r\x031\x08Cg\xb3\xf8...,{'nucl': b'x\x9cM\x93\xd1\r\x031\x08Cg\xb3\xf8...,{'nucl': b'x\x9cM\x93\xd1\r\x031\x08Cg\xb3\xf8...,{'nucl': b'x\x9cE\x93\xd1\r\xc3@\x08Cg\xb3\xf8...,{'nucl': b'x\x9cE\x93\xd1\r\xc3@\x08Cg\xb3\xf8...
OG0003430,"{'nucl': b""x\x9c=\x92\xdb\x11CA\x08Bkc\xf6\x83...",{'nucl': b'x\x9c5\x92\xd1\x11\xc4@\x08Bkc\xf6\...,{'nucl': b'x\x9c5\x92\xd9\x11\xc3@\x0cBk\xd3\x...,{'nucl': b'x\x9c5\x92\xd9\x11\xc3@\x0cBk\xd3\x...,{'nucl': b'x\x9c5\x92\xd9\x11\xc3@\x0cBk\xd3\x...,{'nucl': b'x\x9c5\x92\xd1\x11\xc4@\x08Bkc\xf6\...,{'nucl': b'x\x9c5\x92\xd1\x11\xc4@\x08Bkc\xf6\...,{'nucl': b'x\x9c5\x92\xd1\x11\xc4@\x08Bkc\xf6\...,{'nucl': b'x\x9c5\x92\xd9\x11\xc3@\x0cBk\xd3\x...,{'nucl': b'x\x9c5\x92\xd9\x11\xc3@\x0cBk\xd3\x...,{'nucl': b'x\x9c5\x92\xd1\x11\xc4@\x08Bkc\xf6\...,{'nucl': b'x\x9c=\x92\xd9\r\x041\x08CkC|\xd0\x...,{'nucl': b'x\x9c5\x92\xd9\x11\xc3@\x0cBk\xd3\x...,{'nucl': b'x\x9c5\x92\xd9\x11\xc3@\x0cBk\xd3\x...,{'nucl': b'x\x9c5\x92\xd1\x11\xc4@\x08Bkc\xf6\...,{'nucl': b'x\x9c5\x92\xd1\x11\xc4@\x08Bkc\xf6\...,{'nucl': b'x\x9c5\x92\xd1\x11\xc4@\x08Bkc\xf6\...,{'nucl': b'x\x9c5\x92\xd9\x11\xc3@\x0cBk\xd3\x...,{'nucl': b'x\x9c5\x92\xd9\x11\xc3@\x0cBk\xd3\x...
OG0003429,{'nucl': b'x\x9c=\x92Q\x12\xc40\x08B\xcf\xc6\x...,{'nucl': b'x\x9cE\x92Q\x92\x041\x08B\xcfF\xf9\...,{'nucl': b'x\x9cE\x92\xd1\x11\xc30\x08Cg\xd3\x...,{'nucl': b'x\x9cE\x92\xd1\x11\xc30\x08Cg\xd3\x...,{'nucl': b'x\x9cE\x92\xd1\x11\xc30\x08Cg\xd3\x...,{'nucl': b'x\x9cE\x92Q\x92\x041\x08B\xcfF\xf9\...,{'nucl': b'x\x9cE\x92Q\x92\x041\x08B\xcfF\xf9\...,{'nucl': b'x\x9cE\x92Q\x92\x041\x08B\xcfF\xf9\...,{'nucl': b'x\x9cE\x92\xd1\x11\xc30\x08Cg\xd3\x...,{'nucl': b'x\x9cE\x92\xd1\x11\xc30\x08Cg\xd3\x...,{'nucl': b'x\x9cE\x92Q\x92\x041\x08B\xcfF\xf9\...,"{'nucl': b""x\x9c=\x92\xd1\x11\x041\x08Bkc\xfc\...",{'nucl': b'x\x9cE\x92\xd1\x11\xc30\x08Cg\xd3\x...,{'nucl': b'x\x9cE\x92\xd1\x11\xc30\x08Cg\xd3\x...,{'nucl': b'x\x9cE\x92Q\x92\x041\x08B\xcfF\xf9\...,{'nucl': b'x\x9cE\x92Q\x92\x041\x08B\xcfF\xf9\...,{'nucl': b'x\x9cE\x92Q\x92\x041\x08B\xcfF\xf9\...,{'nucl': b'x\x9cE\x92\xd1\x11\xc30\x08Cg\xd3\x...,{'nucl': b'x\x9cE\x92\xd1\x11\xc30\x08Cg\xd3\x...
OG0003428,"{'nucl': b""x\x9cM\x93\xdbqD1\x0cBkc\xf4A\x03\x...",

In [42]:
# get core ogs
f = open(core_ogs,"r")
ogs = []
for l in f:
    ogs.append(l.replace('\n',''))
f.close()
##print(ogs)

In [43]:
# load roteins ids from the OGs table 
# keep only the core OGs

tab_prot = pd.read_csv(ogs2prot, sep='\t', index_col=0)  
tab_prot = tab_prot.loc[ogs]
tab_prot = tab_prot.rename(columns=lambda x: str(x).replace('_protein','')) # ncbi file names correction
tab_prot = tab_prot.rename(columns=strainsNames2fileNames)
tab_prot

,pe1,pe13,pe15,pe18,pe26,pe27,pe3,pe30,pe39,pe56,pe57,pe61,pe9,to1,to28,to36,to53,to63,to7
Orthogroup,,,,,,,,,,,,,,,,,,,
OG0000211,WP_003261842.1,AIELFPIA_03026,BODOBMNI_02989,NICADNEC_00844,NCAEKIBJ_02782,WP_003261842.1,WP_003261842.1,EENJCLCO_02906,WP_003261842.1,HFPKLCHG_02865,WP_003261842.1,KPJNPNMP_02960,BANHGPCM_02826,JICADCBL_03005,IIHHNEFC_00800,FAFFGKOF_02936,GDEOEPAP_02558,BFOHDJIN_02838,HOLBGOEP_02887
OG0000212,WP_003261911.1,AIELFPIA_01437,BODOBMNI_01473,NICADNEC_02976,NCAEKIBJ_01412,WP_003261911.1,WP_003261911.1,EENJCLCO_01433,WP_003261911.1,HFPKLCHG_01383,WP_003261911.1,KPJNPNMP_01417,BANHGPCM_01414,JICADCBL_01454,IIHHNEFC_02313,FAFFGKOF_01401,GDEOEPAP_01326,BFOHDJIN_01321,HOLBGOEP_01408
OG0000213,WP_003261929.1,AIELFPIA_01452,BODOBMNI_01488,NICADNEC_02991,NCAEKIBJ_01427,WP_003261929.1,WP_003261929.1,EENJCLCO_01448,WP_003261929.1,HFPKLCHG_01398,WP_003261929.1,KPJNPNMP_01432,BANHGPCM_01429,JICADCBL_01469,IIHHNEFC_02298,FAFFGKOF_01416,GDEOEPAP_01341,BFOHDJIN_01336,HOLBGOEP_01423
OG0000214,WP_003261939.1,AIELFPIA_01459,BODOBMNI_01495,NICADNEC_02998,NCAEKIBJ_01434,WP_003261939.1,WP_003261939.1,EENJCLCO_01455,WP_003261939.1,HFPKLCHG_01405,WP_003261939.1,KPJNPNMP_01439,BANHGPCM_01436,JICADCBL_01476,IIHHNEFC_02291,FAFFGKOF_01423,GDEOEPAP_01348,BFOHDJIN_01343,HOLBGOEP_01430
OG0000215,WP_003261941.1,AIELFPIA_01461,BODOBMNI_01497,NICADNEC_03000,NCAEKIBJ_01436,WP_003261941.1,WP_003261941.1,EENJCLCO_01457,WP_003261941.1,HFPKLCHG_01407,WP_003261941.1,KPJNPNMP_01441,BANHGPCM_01438,JICADCBL_01478,IIHHNEFC_02289,FAFFGKOF_01425,GDEOEPAP_01350,BFOHDJIN_01345,HOLBGOEP_01432
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
OG0004004,WP_225891747.1,AIELFPIA_04069,BODOBMNI_04036,NICADNEC_03941,NCAEKIBJ_04729,WP_081365349.1,WP_052331360.1,EENJCLCO_03981,WP_165858710.1,HFPKLCHG_03918,WP_167798393.1,KPJNPNMP_04008,BANHGPCM_03905,JICADCBL_04053,IIHHNEFC_03963,FAFFGKOF_04015,GDEOEPAP_03567,BFOHDJIN_03889,HOLBGOEP_03937
OG0004005,WP_225891750.1,AIELFPIA_03907,BODOBMNI_03873,NICADNEC_03778,NCAEKIBJ_04891,WP_071507083.1,WP_064047918.1,EENJCLCO_03818,WP_071507083.1,HFPKLCHG_03753,WP_071507083.1,KPJNPNMP_03846,BANHGPCM_03744,JICADCBL_03889,IIHHNEFC_03802,FAFFGKOF_03854,GDEOEPAP_03395,BFOHDJIN_03728,HOLBGOEP_03775
OG0004006,WP_225891752.1,AIELFPIA_03879,BODOBMNI_03845,NICADNEC_03750,NCAEKIBJ_04921,WP_038937612.1,WP_038937612.1,EENJCLCO_03790,WP_089190982.1,HFPKLCHG_03721,WP_071507970.1,KPJNPNMP_03814,BANHGPCM_03715,JICADCBL_03859,IIHHNEFC_03773,FAFFGKOF_03825,GDEOEPAP_04826,BFOHDJIN_03696,HOLBGOEP_03743


In [44]:
genomes_for_og = list(tab_prot.columns) 

#for x in ['pe1', 'pe3', 'pe27', 'pe57', 'pe39']:
#    genomes_for_og.remove(x)
#genomes_for_og

# Get Nucleotide sequences + corresponding positions in genome

# Load single proteins alignments and extract gaps

In [45]:
gaps2tab = pd.DataFrame(columns=tab_prot.columns, index=tab_prot.index)

len_counter = 0

for og in ogs:
    pmsa = path_p_aln + '/' +og + '.fa'
    genome_ids = []
    #if len_counter >= 395140:
      #  print(og, '-----')
        #len_counter = 0
            
    for record in SeqIO.parse(pmsa, "fasta"):  
        #print(pmsa)
        
        header = record.id.split('_')
        
        if record.id.count(header[-2])>1:
            # martinique header structure
            if record.id.startswith(header[-2]):
                genome_id=header[-2]
            else:
                genome_id=header[-3]+'_'+header[-2]
            protein_id = record.id.replace('_'+genome_id,'')
        else:
            # prokka/ncbi genomes header structure 
            protein_id = header[-2]+'_'+header[-1]
            genome_id = strainsNames2fileNames[record.id.replace(protein_id, '').replace('protein_', '')[:-1].replace('_1_', '.1_')]
            #genome_ids.append(genome_id)
            #prot_db.loc[og, genome_id]['og'] = og
        
        #print(pmsa)
        #print(protein_id, genome_id)
        
        # GAPS PROFILE of aa alignment  
        gaps_bool = []
        
        #if og == 'OG0000732' and genome_id == 'to53':
            #print('---', len(record.seq) )
        
        for aa in record.seq:
            if aa == '-':
                gaps_bool.append(1) 
                gaps_bool.append(1) 
                gaps_bool.append(1) 
            else:
                gaps_bool.append(0) 
                gaps_bool.append(0) 
                gaps_bool.append(0)
        
        
        #if genome_id == 'to53':
        #    print(gaps_bool)
        #    print('to53',record.seq)
        #if genome_id == 'to7':
        #    print(gaps_bool)
        #    print('to7', record.seq)
        prot_db.loc[og, genome_id]['aln_gaps'] = gaps_bool
        #prot_db.loc[og, genome_id]['og'] = og
        gaps2tab.loc[og][genome_id] = gaps_bool
        
        #if len(record.seq)*3 != len(prot_db.loc[protein_id, genome_id]['nucl']):
            #print(protein_id, genome_id)
            #print('ffffff',len(record.seq)*3, len(prot_db.loc[protein_id, genome_id]['nucl']))
    #print(og, genome_ids)
gaps2tab

,pe1,pe13,pe15,pe18,pe26,pe27,pe3,pe30,pe39,pe56,pe57,pe61,pe9,to1,to28,to36,to53,to63,to7
Orthogroup,,,,,,,,,,,,,,,,,,,
OG0000211,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
OG0000212,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
OG0000213,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
OG0000214,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

sss='23-10BR_23-10BR_01627'
sss='CFBP6783_2_CFBP6783_2_02036'
header = sss.split('_')

if sss.count(header[-2])>1:
    if sss.startswith(header[-2]):
        genome_id=header[-2]
    else:
        genome_id=header[-3]+'_'+header[-2]
    protein_id = sss.replace('_'+genome_id,'')
print(protein_id, genome_id)        

import ast
d = ast.literal_eval(prot_db.loc[protein_id, genome_id])
d
d['og'] = og
prot_db.loc[protein_id, genome_id] = d

In [46]:
ogs_to_drop_from_core = []
for g in genomes_for_og:
    for og in ogs:
        
        msa_l = len(prot_db.loc[og, g]['aln_gaps'])
        seq_l = len(zlib.decompress(prot_db.loc[og, g]['nucl']).decode())
        gap_l = gaps2tab.loc[og, g].count(1)
        
        if seq_l + gap_l != msa_l:
            print('Length error in:', g, og, ': MSA length:', msa_l, 'protein length:' ,seq_l)
            ogs_to_drop_from_core.append(og)
            
tab_prot = tab_prot.drop(ogs_to_drop_from_core)
gaps2tab = gaps2tab.drop(ogs_to_drop_from_core)
ogs = [i for i in ogs if i not in ogs_to_drop_from_core]

Length error in: pe1 OG0002711 : MSA length: 1923 protein length: 1953
Length error in: pe1 OG0003717 : MSA length: 939 protein length: 866
Length error in: pe1 OG0003971 : MSA length: 2421 protein length: 2547
Length error in: pe1 OG0003985 : MSA length: 2640 protein length: 2874
Length error in: pe1 OG0003994 : MSA length: 6624 protein length: 6873
Length error in: pe1 OG0004007 : MSA length: 4122 protein length: 4311
Length error in: pe13 OG0003025 : MSA length: 1620 protein length: 1635
Length error in: pe18 OG0002939 : MSA length: 3993 protein length: 4026
Length error in: pe3 OG0003980 : MSA length: 7122 protein length: 6771
Length error in: pe57 OG0003497 : MSA length: 2859 protein length: 2850
Length error in: pe57 OG0003778 : MSA length: 1506 protein length: 1509
Length error in: pe57 OG0003823 : MSA length: 2337 protein length: 2343
Length error in: pe57 OG0003876 : MSA length: 2736 protein length: 2739
Length error in: pe57 OG0003917 : MSA length: 7590 protein length: 7491
L

In [48]:
prot_db=prot_db.loc[ogs]

# Genome-based solution
+ for each genome grab the cols of the core OG table
+ concatenate the corresponding nucleotides from the GENOME_PROTEIN_DB

## TODO length of nucleotides is larger than length of translation*3 ?!

In [49]:
g2nucl = {}
g2pos = {}
g2chr = {}
g2strand = {}
g2acc = {}
g2gaps = {}
g2og = {}

for g in genomes_for_og:
    print('Genome', g)
    
    # get corresponding nucleotide positions 
    g2pos[g] = []
    # get corresponding chr ids 
    g2chr[g] = []
    # get corresponding strands
    g2strand[g] = []
    # get protein accession for each genome position
    g2acc[g] = []
    # get gaps profile
    g2gaps[g] = []
    # debug
    g2og[g] = []
    
    # get nucleotide sequences for all the core og proteins per genome 
    nucl_per_genome = []
    #print(prot_db.loc[tab_prot[g].values, g])
    
    for i in prot_db[g]: 
        
        nucl_per_genome.append(zlib.decompress(i['nucl']).decode())
        #g2pos[g].extend(i['location'])
        #print('----',g, )
        #g2og[g].append(i['og'])
        
        for counter in range(len(i['aln_gaps'])):
            #g2chr[g].append(i['chr'])
            #g2strand[g].append(i['location'].strand)
            g2acc[g].append(i['acc'])
        g2gaps[g].extend(i['aln_gaps'])
    
    nucl_per_genome = "".join(nucl_per_genome)
    g2nucl[g] = nucl_per_genome
    print('Exported sequence length NO GAPS', len(nucl_per_genome))
    print('Exported alignment WITH GAPS', len(g2gaps[g]))
    print('Exported protein accessions', len(g2acc[g]))
    print('Exported numeric positions', len(g2pos[g]))
    print('Exported chromosomal positions', len(g2chr[g]))
    print('Exported strands orientations', len(g2strand[g]))
    print()
    
    ####### debug 
    #g2nucl[g] = g2nucl[g][:395150]
    #g2gaps[g] = g2gaps[g][:395150]
    #g2acc[g] = g2acc[g][:395150]
    

Genome pe1
Exported sequence length NO GAPS 3738591
Exported alignment WITH GAPS 3761544
Exported protein accessions 3761544
Exported numeric positions 0
Exported chromosomal positions 0
Exported strands orientations 0

Genome pe13
Exported sequence length NO GAPS 3740577
Exported alignment WITH GAPS 3761544
Exported protein accessions 3761544
Exported numeric positions 0
Exported chromosomal positions 0
Exported strands orientations 0

Genome pe15
Exported sequence length NO GAPS 3739377
Exported alignment WITH GAPS 3761544
Exported protein accessions 3761544
Exported numeric positions 0
Exported chromosomal positions 0
Exported strands orientations 0

Genome pe18
Exported sequence length NO GAPS 3737541
Exported alignment WITH GAPS 3761544
Exported protein accessions 3761544
Exported numeric positions 0
Exported chromosomal positions 0
Exported strands orientations 0

Genome pe26
Exported sequence length NO GAPS 3741738
Exported alignment WITH GAPS 3761544
Exported protein accessions

# MSA repeats problem 

some proteins sequence of the OrthoFinder may have long repeat sequences, which are not present in the orthologs. In the resulting MSA produced by OrthoFinder the repeated sequences are skipped:

e.g. 
+ original input: <br>
\> p1 <br>
ABCXYZ <br>
\> p2 <br>
ABCABCXYZ <br>
+ MSA output:<br>
\> p1 <br>
ABCXYZ <br>
\> p2 <br>
ABCXYZ <br>

This results in large shifts in the final nucleotides MSA <br>

Examples from to53

OG: OG0000732; gene: DEOEPAP_01560
MNSKTDRPAEGAGATADRAAREAAVARALECLADHAGDHQTLTGEPLVSHARGTVAILEGLRVDAPSLQAAALFLLPILATDNEKAIEPAFGGEVARLVHDVRQLLRIGAIAGMASPAEAGVTRKNEAEARRAQVEALRKMLLAFAQDIRVVLIRLASRLQSLRWLAQNKLPAPEGMARETLDIYAPLANRLGIWQLKWELEDLGFRFEDPDTYKRIARLLDEKRIEREQFIGEAIARLQQTLRDAGIHAEVSGRPKHIYSIWRKMRGKDLDFADLYDVRAFRVIVDDIKDCYTVLGFVHHMWQPIPKEFDDYISRPKTNGYKSLHTVNSKTDRPAEGAGATADRAAREAAVARALECLADHAGDHQTLTGEPLVSHARGTVAILEGLRVDAPSLQAAALFLLPILATDNEKAIEPAFGGEVARLVHDVRQLLRIGAIAGMASPAEAGVTRKNEAEARRAQVEALRKMLLAFAQDIRVVLIRLASRLQSLRWLAQNKLPAPEGMARETLDIYAPLANRLGIWQLKWELEDLGFRFEDPDTYKRIARLLDEKRIEREQFIGEAIARLQQTLRDAGIHAEVSGRPKHIYSIWRKMRGKDLDFADLYDVRAFRVIVDDIKDCYTVLGFVHHMWQPIPKEFDDYISRPKTNGYKSLHTVVIGDDGRALEVQIRTREMHHFAEYGVAAHWRYKEAGSKGYAGQFSASERYDEKIAWLRQLLAWKDDAEHTVAQEDSPWEQLKHTELDDHIYVLTPQARVIALPQGATPVDFAYYLHSDLGHRCRGARVDGTMVPLNTPLKNGQTVEIVTVKQGGPSRDWLNPELHYLASNRARAKVRAWFNALELEETLAQGRILIDKTLQREGKTAVNLEELAAKLGFKTPDELYAMVAKDEFSLRHVEAVLRTDGAPPAPADDEASITKKSRATSVARGAKSGVLVVGVDSLLTQMSRCCKPAPPDLIVGFVTRGRGVSIHRQNCATFQQLAARAPGRVIQTEWGNRGEAVYPVDIHVEALDRQGLLRDISEILSREKINVTGVRTLSSKGVAKMQFTAEVSEATQLQRALILIEEVTGVLSARRK
VNSKTDRPAEGAGATADRAAREAAVARALECLADHAGDHQTLTGEPLVSHARGTVAILEGLRVDAPSLQAAALFLLPILATDNEKAIEPAFGGEVARLVHDVRQLLRIGAIAGMASPAEAGVTRKNEAEARRAQVEALRKMLLAFAQDIRVVLIRLASRLQSLRWLAQNKLPAPEGMARETLDIYAPLANRLGIWQLKWELEDLGFRFEDPDTYKRIARLLDEKRIEREQFIGEAIARLQQTLRDAGIHAEVSGRPKHIYSIWRKMRGKDLDFADLYDVRAFRVIVDDIKDCYTVLGFVHHMWQPIPKEFDDYISRPKTNGYKSLHT                                                                                                                                                                                                                                                                                                                                       VVIGDDGRALEVQIRTREMHHFAEYGVAAHWRYKEAGSKGYAGQFSASERYDEKIAWLRQLLAWKDDAEHTVAQEDSPWEQLKHTELDDHIYVLTPQARVIALPQGATPVDFAYYLHSDLGHRCRGARVDGTMVPLNTPLKNGQTVEIVTVKQGGPSRDWLNPELHYLASNRARAKVRAWFNALELEETLAQGRILIDKTLQREGKTAVNLEELAAKLGFKTPDELYAMVAKDEFSLRHVEAVLRTDGAPPAPADDEASITKKSRATSVARGAKSGVLVVGVDSLLTQMSRCCKPAPPDLIVGFVTRGRGVSIHRQNCATFQQLAARAPGRVIQTEWGNRGEAVYPVDIHVEALDRQGLLRDISEILSREKINVTGVRTLSSKGVAKMQFTAEVSEATQLQRALILIEEVTGVLSARRK

OG: OG0002212; gene: GDEOEPAP_02088
MTRVDLRSAKPDYDGPLLRLDDLSVRFEAEHNEWIEAVRHVSFDLHAGERFALVGESGSGKTVTALSIMRLLADAHYSGRILFEGRDLLAASERQMRGLRGADVAMVFQEPMTALNPLYTIGNQIVETLELHEGLDKRAAKARAIALLEHTGIAEAPRRFDAFPHQLSGGQRQRAMIAMALACSPKLLVADEPTTALDVTVRMQILQLLRDLQAEYGMAVMLITHDLNMVRAFAERVGVMERGVLVETGETATVFSAPQHPYTVRLLESRPQRDVLPLVPLAPVLLEADQLTVTYDRHRPGFAGWFRADPFTAVDQVSLQLREGETLGIVGESGSGKTTLAQTLLGLQRPKAGDLRFLGSSLLGQTRDERRAVRARMQVVFQDPFGSLSPRMTIEQIVGEGLALHQPHVNDAERRQRVIEALREVGLDRTALGRYPHEFSGGQRQRIAIARVLILKPQVLVLDEPTSALDVSIQQQVLSLLSALQKKYNLSYLFISHDLAVIRAMSHRVMVMKDGKVVEAGDTEAVLASPSHPYTCKLMAAAALKPLQPLQPPG
MTRVDLRSAKPDYDGPLLRLDDLSVRFEAEHNEWIEAVRHVSFDLHAGERFALVGESGSGKTVTALSIMRLLADAHYSGRILFEGRDLLAASERQMRGLRGADVAMVFQEPMTALNPLYTIGNQIVETLELHEGLDKRAAKARAIALLEHTGIAEAPRRFDAFPHQLSGGQRQRAMIAMALACSPKLLVADEPTTALDVTVRMQILQLLRDLQAEYGMAVMLITHDLNMVRAFAERVGVMERGVLVETGETATVFSAPQHPYTVRLLESRPQRDVLPLVPLAPVLLEADQLTVTYDRHRPGFAGWFRADPFTAVDQVSLQLREGETLGIVGESGSGKTTLAQTLLGLQRPKAGDLRFLGSSLLGQTRDERRAVRARMQVVFQDPFGSLSPRMTIEQIVGEGLALHQPHVNDAERRQRVIEALREVGLDRTALGRYPHEFSGGQRQRIAIARVLILKPQVLVLDEPTSALDVSIQQQVLSLLSALQKKYNLSYLFISHDLAVIRAMSHRVMVMKDGKVVEAGDTEAVLASPSHPYTCKLMAAAALKPLQ   PPG

OG: OG0003692; gene: GDEOEPAP_04673
MRHFPTLHSVALIPSGAAITRAAVTGHARHPRAPVRNALGGTVLLGTFAAAIPAFAAPAGGTVTSGSGSIGQNGTTTTITQTSSRLAIDWSAFGIQAGETVNFIQPGAGAIALNRVTGHEATSIFGSLNANGTVFILNPNGVLFGGGAQVNVGGLVASTLGLSNADFEAGRYALSGGSTAGVTNQGTITVPSGGKVALIANSVDNAGNISAPRGSVLLAGAGNVTLTLADGSPLGYTISAGAARTLVNNGGMIVADGGRVVLTARGLDSLSESVVNTTGVVRARTVGNNQGTIELIGDPMAGLTQVSGQIDASAPDGGGNGGSVKVLGTKVGVFDGARIDVSGMAGGGTALIGGNAQGAGPEPNATATYVAPTAAIDASAIRQGNGGRLIVWGTDVANVHGSLSANGGAQGGNGGHVETSGHALDTDGIAVSVAGGGGG  TGGLWLLDPYNVTLSAGTQTGGAFSGNVWTPSASGSLVNTSSIQSLLNSGGNVTITTTGAGTQEGNIAINGSISKTAGGNASLTLIADGRITTNASSGTHRTITSTSGTLDVSMTARATTSAASTSGINLRYVDINANGGNISATASGAQSGTAAALSLQNSSWSTTGAGNISLSGTLPDNGNSQGVFLNANTLTTASGSITVSGTSGGIATITGTNSNGAPLFSTSNIGVYLAGGNTLRSTGGGAIALTGTATGATATWAGSGVTLSALDSLSTSGAVTISGTASNPVSSTYRNQLSTVNIAGTGSNAASLTGGTVTITGTNSVVGNASSTSNGNAAVKLDGKVNLTATAGPINIAGSNAGGDGVWGSGSGAVTMSAPASSSITISARSLDSVSGYSGFYIGGGSATLTFATAAPVSIAAESLVGARKAFWNKGGLTVPGNLSIVTTGGAIADDTTHGGYFHVTGSTSIDSSGAGNTVSLTNSGNTFAGAISLNAGDTTLASSTSLTLDSSTVNGALNLTAPGITQSGAVSAAGTATLNGGNAAITLTNDGNAFNALAVTGAAGVNVVNANALSVTGVTSTGAVRFTTLSGDLATRGNWSISGGDLTLSAGAGNTRGTASGGNVDSQATLAVDAGRTITVYSGAVGSTVLGGTLAGRAAAGSGNFRYNRQDGDAPGADGIGDGATYVMYRERPTVTVTPTDAANTKVYDGGGASDPALAYTIVGQVNGDSATQILSGSLARAAGQNAGRYAISQGTLADQLGYQTVLDVGHAFSITPAPLLITANGASRTAGQDNPAFSATFAGFVGGQTAQTADLRGTLRLLTSADAGSPAGSYAIVPDGISSNNYSIRYAAGTLSVLPASPAPMVAVPGEPSYMNAVIDMQARIRAAGLASAPEPAERSATGWRTTTATFRQDDTASDNVPGNRLAAQE
MRHFPTLHSVALIPSGAAITRAAVTGHARHPRAPVRNALGGTVLLGTFAAAIPAFAAPAGGTVTSGSGSIGQNGTTTTITQTSSRLAIDWSAFGIQAGETVNFIQPGAGAIALNRVTGHEATSIFGSLNANGTVFILNPNGVLFGGGAQVNVGGLVASTLGLSNADFEAGRYALSGGSTAGVTNQGTITVPSGGKVALIANSVDNAGNISAPRGSVLLAGAGNVTLTLADGSPLGYTISAGAARTLVNNGGMIVADGGRVVLTARGLDSLSESVVNTTGVVRARTVGNNQGTIELIGDPMAGLTQVSGQIDASAPDGGGNGGSVKVLGTKVGVFDGARIDVSGMAGGGTALIGGNAQGAGPEPNATATYVAPTAAIDASAIRQGNGGRLIVWGTDVANVHGSLSANGGAQGGNGGHVETSGHALDTDGIAVSVAGGGGGGGTGGLWLLDPYNVTLSAGTQTGGAFSGNVWTPSASGSLVNTSSIQSLLNSGGNVTITTTGAGTQEGNIAINGSISKTAGGNASLTLIADGRITTNASSGTHRTITSTSGTLDVSMTARATTSAASTSGINLRYVDINANGGNISATASGAQSGTAAALSLQNSSWSTTGAGNISLSGTLPDNGNSQGVFLNANTLTTASGSITVSGTSGGIATITGTNSNGAPLFSTSNIGVYLAGGNTLRSTGGGAIALTGTATGATATWAGSGVTLSALDSLSTSGAVTISGTASNPVSSTYRNQLSTVNIAGTGSNAASLTGGTVTITGTNSVVGNASSTSNGNAAVKLDGKVNLTATAGPINIAGSNAGGDGVWGSGSGAVTMSAPASSSITISARSLDSVSGYSGFYIGGGSATLTFATAAPVSIAAESLVGARKAFWNKGGLTVPGNLSIVTTGGAIADDTTHGGYFHVTGSTSIDSSGAGNTVSLTNSGNTFAGAISLNAGDTTLASSTSLTLDSSTVNGALNLTAPGITQSGAVSAAGTATLNGGNAAITLTNDGNAFNALAVTGAAGVNVVNANALSVTGVTSTGAVRFTTLSGDLATRGNWSISGGDLTLSAGAGNTRGTASGGNVDSQATLAVDAGRTITVYSGAVGSTVLGGTLAGRAAAGSGNFRYNRQDGDAPGADGIGDGATYVMYRERPTVTVTPTDAANTKVYDGGGASDPALAYTIVGQVNGDSATQILSGSLARAAGQNAGRYAISQGTLADQLGYQTVLDVGHAFSITPAPLLITANGASRTAGQDNPAFSATFAGFVGGQTAQTADLRGTLRLLTSADAGSPAGSYAIVPDGISSNNYSIRYAAGTLSVLPASPAPMVAVPGEPSYMNAVIDMQARIRAAGLASAPEPAERSATGWRTTTATFRQDDTASDNVPGNRLAAQE

In [50]:
import difflib

s_nucl ="MTRVDLRSAKPDYDGPLLRLDDLSVRFEAEHNEWIEAVRHVSFDLHAGERFALVGESGSGKTVTALSIMRLLADAHYSGRILFEGRDLLAASERQMRGLRGADVAMVFQEPMTALNPLYTIGNQIVETLELHEGLDKRAAKARAIALLEHTGIAEAPRRFDAFPHQLSGGQRQRAMIAMALACSPKLLVADEPTTALDVTVRMQILQLLRDLQAEYGMAVMLITHDLNMVRAFAERVGVMERGVLVETGETATVFSAPQHPYTVRLLESRPQRDVLPLVPLAPVLLEADQLTVTYDRHRPGFAGWFRADPFTAVDQVSLQLREGETLGIVGESGSGKTTLAQTLLGLQRPKAGDLRFLGSSLLGQTRDERRAVRARMQVVFQDPFGSLSPRMTIEQIVGEGLALHQPHVNDAERRQRVIEALREVGLDRTALGRYPHEFSGGQRQRIAIARVLILKPQVLVLDEPTSALDVSIQQQVLSLLSALQKKYNLSYLFISHDLAVIRAMSHRVMVMKDGKVVEAGDTEAVLASPSHPYTCKLMAAAALKPLQPLQPPG"
s_msa = "MTRVDLRSAKPDYDGPLLRLDDLSVRFEAEHNEWIEAVRHVSFDLHAGERFALVGESGSGKTVTALSIMRLLADAHYSGRILFEGRDLLAASERQMRGLRGADVAMVFQEPMTALNPLYTIGNQIVETLELHEGLDKRAAKARAIALLEHTGIAEAPRRFDAFPHQLSGGQRQRAMIAMALACSPKLLVADEPTTALDVTVRMQILQLLRDLQAEYGMAVMLITHDLNMVRAFAERVGVMERGVLVETGETATVFSAPQHPYTVRLLESRPQRDVLPLVPLAPVLLEADQLTVTYDRHRPGFAGWFRADPFTAVDQVSLQLREGETLGIVGESGSGKTTLAQTLLGLQRPKAGDLRFLGSSLLGQTRDERRAVRARMQVVFQDPFGSLSPRMTIEQIVGEGLALHQPHVNDAERRQRVIEALREVGLDRTALGRYPHEFSGGQRQRIAIARVLILKPQVLVLDEPTSALDVSIQQQVLSLLSALQKKYNLSYLFISHDLAVIRAMSHRVMVMKDGKVVEAGDTEAVLASPSHPYTCKLMAAAALKPLQPPG"

s_msa = "MRHFPTLHSVALIPSGAAITRAAVTGHARHPRAPVRNALGGTVLLGTFAAAIPAFAAPAGGTVTSGSGSIGQNGTTTTITQTSSRLAIDWSAFGIQAGETVNFIQPGAGAIALNRVTGHEATSIFGSLNANGTVFILNPNGVLFGGGAQVNVGGLVASTLGLSNADFEAGRYALSGGSTAGVTNQGTITVPSGGKVALIANSVDNAGNISAPRGSVLLAGAGNVTLTLADGSPLGYTISAGAARTLVNNGGMIVADGGRVVLTARGLDSLSESVVNTTGVVRARTVGNNQGTIELIGDPMAGLTQVSGQIDASAPDGGGNGGSVKVLGTKVGVFDGARIDVSGMAGGGTALIGGNAQGAGPEPNATATYVAPTAAIDASAIRQGNGGRLIVWGTDVANVHGSLSANGGAQGGNGGHVETSGHALDTDGIAVSVAGGGGGTGGLWLLDPYNVTLSAGTQTGGAFSGNVWTPSASGSLVNTSSIQSLLNSGGNVTITTTGAGTQEGNIAINGSISKTAGGNASLTLIADGRITTNASSGTHRTITSTSGTLDVSMTARATTSAASTSGINLRYVDINANGGNISATASGAQSGTAAALSLQNSSWSTTGAGNISLSGTLPDNGNSQGVFLNANTLTTASGSITVSGTSGGIATITGTNSNGAPLFSTSNIGVYLAGGNTLRSTGGGAIALTGTATGATATWAGSGVTLSALDSLSTSGAVTISGTASNPVSSTYRNQLSTVNIAGTGSNAASLTGGTVTITGTNSVVGNASSTSNGNAAVKLDGKVNLTATAGPINIAGSNAGGDGVWGSGSGAVTMSAPASSSITISARSLDSVSGYSGFYIGGGSATLTFATAAPVSIAAESLVGARKAFWNKGGLTVPGNLSIVTTGGAIADDTTHGGYFHVTGSTSIDSSGAGNTVSLTNSGNTFAGAISLNAGDTTLASSTSLTLDSSTVNGALNLTAPGITQSGAVSAAGTATLNGGNAAITLTNDGNAFNALAVTGAAGVNVVNANALSVTGVTSTGAVRFTTLSGDLATRGNWSISGGDLTLSAGAGNTRGTASGGNVDSQATLAVDAGRTITVYSGAVGSTVLGGTLAGRAAAGSGNFRYNRQDGDAPGADGIGDGATYVMYRERPTVTVTPTDAANTKVYDGGGASDPALAYTIVGQVNGDSATQILSGSLARAAGQNAGRYAISQGTLADQLGYQTVLDVGHAFSITPAPLLITANGASRTAGQDNPAFSATFAGFVGGQTAQTADLRGTLRLLTSADAGSPAGSYAIVPDGISSNNYSIRYAAGTLSVLPASPAPMVAVPGEPSYMNAVIDMQARIRAAGLASAPEPAERSATGWRTTTATFRQDDTASDNVPGNRLAAQE"
s_nucl =  "MRHFPTLHSVALIPSGAAITRAAVTGHARHPRAPVRNALGGTVLLGTFAAAIPAFAAPAGGTVTSGSGSIGQNGTTTTITQTSSRLAIDWSAFGIQAGETVNFIQPGAGAIALNRVTGHEATSIFGSLNANGTVFILNPNGVLFGGGAQVNVGGLVASTLGLSNADFEAGRYALSGGSTAGVTNQGTITVPSGGKVALIANSVDNAGNISAPRGSVLLAGAGNVTLTLADGSPLGYTISAGAARTLVNNGGMIVADGGRVVLTARGLDSLSESVVNTTGVVRARTVGNNQGTIELIGDPMAGLTQVSGQIDASAPDGGGNGGSVKVLGTKVGVFDGARIDVSGMAGGGTALIGGNAQGAGPEPNATATYVAPTAAIDASAIRQGNGGRLIVWGTDVANVHGSLSANGGAQGGNGGHVETSGHALDTDGIAVSVAGGGGGGGTGGLWLLDPYNVTLSAGTQTGGAFSGNVWTPSASGSLVNTSSIQSLLNSGGNVTITTTGAGTQEGNIAINGSISKTAGGNASLTLIADGRITTNASSGTHRTITSTSGTLDVSMTARATTSAASTSGINLRYVDINANGGNISATASGAQSGTAAALSLQNSSWSTTGAGNISLSGTLPDNGNSQGVFLNANTLTTASGSITVSGTSGGIATITGTNSNGAPLFSTSNIGVYLAGGNTLRSTGGGAIALTGTATGATATWAGSGVTLSALDSLSTSGAVTISGTASNPVSSTYRNQLSTVNIAGTGSNAASLTGGTVTITGTNSVVGNASSTSNGNAAVKLDGKVNLTATAGPINIAGSNAGGDGVWGSGSGAVTMSAPASSSITISARSLDSVSGYSGFYIGGGSATLTFATAAPVSIAAESLVGARKAFWNKGGLTVPGNLSIVTTGGAIADDTTHGGYFHVTGSTSIDSSGAGNTVSLTNSGNTFAGAISLNAGDTTLASSTSLTLDSSTVNGALNLTAPGITQSGAVSAAGTATLNGGNAAITLTNDGNAFNALAVTGAAGVNVVNANALSVTGVTSTGAVRFTTLSGDLATRGNWSISGGDLTLSAGAGNTRGTASGGNVDSQATLAVDAGRTITVYSGAVGSTVLGGTLAGRAAAGSGNFRYNRQDGDAPGADGIGDGATYVMYRERPTVTVTPTDAANTKVYDGGGASDPALAYTIVGQVNGDSATQILSGSLARAAGQNAGRYAISQGTLADQLGYQTVLDVGHAFSITPAPLLITANGASRTAGQDNPAFSATFAGFVGGQTAQTADLRGTLRLLTSADAGSPAGSYAIVPDGISSNNYSIRYAAGTLSVLPASPAPMVAVPGEPSYMNAVIDMQARIRAAGLASAPEPAERSATGWRTTTATFRQDDTASDNVPGNRLAAQE"

s_nucl ="MNSKTDRPAEGAGATADRAAREAAVARALECLADHAGDHQTLTGEPLVSHARGTVAILEGLRVDAPSLQAAALFLLPILATDNEKAIEPAFGGEVARLVHDVRQLLRIGAIAGMASPAEAGVTRKNEAEARRAQVEALRKMLLAFAQDIRVVLIRLASRLQSLRWLAQNKLPAPEGMARETLDIYAPLANRLGIWQLKWELEDLGFRFEDPDTYKRIARLLDEKRIEREQFIGEAIARLQQTLRDAGIHAEVSGRPKHIYSIWRKMRGKDLDFADLYDVRAFRVIVDDIKDCYTVLGFVHHMWQPIPKEFDDYISRPKTNGYKSLHTVNSKTDRPAEGAGATADRAAREAAVARALECLADHAGDHQTLTGEPLVSHARGTVAILEGLRVDAPSLQAAALFLLPILATDNEKAIEPAFGGEVARLVHDVRQLLRIGAIAGMASPAEAGVTRKNEAEARRAQVEALRKMLLAFAQDIRVVLIRLASRLQSLRWLAQNKLPAPEGMARETLDIYAPLANRLGIWQLKWELEDLGFRFEDPDTYKRIARLLDEKRIEREQFIGEAIARLQQTLRDAGIHAEVSGRPKHIYSIWRKMRGKDLDFADLYDVRAFRVIVDDIKDCYTVLGFVHHMWQPIPKEFDDYISRPKTNGYKSLHTVVIGDDGRALEVQIRTREMHHFAEYGVAAHWRYKEAGSKGYAGQFSASERYDEKIAWLRQLLAWKDDAEHTVAQEDSPWEQLKHTELDDHIYVLTPQARVIALPQGATPVDFAYYLHSDLGHRCRGARVDGTMVPLNTPLKNGQTVEIVTVKQGGPSRDWLNPELHYLASNRARAKVRAWFNALELEETLAQGRILIDKTLQREGKTAVNLEELAAKLGFKTPDELYAMVAKDEFSLRHVEAVLRTDGAPPAPADDEASITKKSRATSVARGAKSGVLVVGVDSLLTQMSRCCKPAPPDLIVGFVTRGRGVSIHRQNCATFQQLAARAPGRVIQTEWGNRGEAVYPVDIHVEALDRQGLLRDISEILSREKINVTGVRTLSSKGVAKMQFTAEVSEATQLQRALILIEEVTGVLSARRK"
s_msa = "VNSKTDRPAEGAGATADRAAREAAVARALECLADHAGDHQTLTGEPLVSHARGTVAILEGLRVDAPSLQAAALFLLPILATDNEKAIEPAFGGEVARLVHDVRQLLRIGAIAGMASPAEAGVTRKNEAEARRAQVEALRKMLLAFAQDIRVVLIRLASRLQSLRWLAQNKLPAPEGMARETLDIYAPLANRLGIWQLKWELEDLGFRFEDPDTYKRIARLLDEKRIEREQFIGEAIARLQQTLRDAGIHAEVSGRPKHIYSIWRKMRGKDLDFADLYDVRAFRVIVDDIKDCYTVLGFVHHMWQPIPKEFDDYISRPKTNGYKSLHTVVIGDDGRALEVQIRTREMHHFAEYGVAAHWRYKEAGSKGYAGQFSASERYDEKIAWLRQLLAWKDDAEHTVAQEDSPWEQLKHTELDDHIYVLTPQARVIALPQGATPVDFAYYLHSDLGHRCRGARVDGTMVPLNTPLKNGQTVEIVTVKQGGPSRDWLNPELHYLASNRARAKVRAWFNALELEETLAQGRILIDKTLQREGKTAVNLEELAAKLGFKTPDELYAMVAKDEFSLRHVEAVLRTDGAPPAPADDEASITKKSRATSVARGAKSGVLVVGVDSLLTQMSRCCKPAPPDLIVGFVTRGRGVSIHRQNCATFQQLAARAPGRVIQTEWGNRGEAVYPVDIHVEALDRQGLLRDISEILSREKINVTGVRTLSSKGVAKMQFTAEVSEATQLQRALILIEEVTGVLSARRK"

#s1-s2
output_list = [li for li in difflib.ndiff(s_nucl, s_msa) if li[0] != ' ']
print(output_list)

repeat_seq = ''.join([i[-1] for i in output_list if i[0] == '-'])
print('rep:', repeat_seq)

# count diff in nucl string 
print(s_nucl.count(repeat_seq[1:]))
print(s_msa.count(repeat_seq[1:]))

['- M', '- N', '- S', '- K', '- T', '- D', '- R', '- P', '- A', '- E', '- G', '- A', '- G', '- A', '- T', '- A', '- D', '- R', '- A', '- A', '- R', '- E', '- A', '- A', '- V', '- A', '- R', '- A', '- L', '- E', '- C', '- L', '- A', '- D', '- H', '- A', '- G', '- D', '- H', '- Q', '- T', '- L', '- T', '- G', '- E', '- P', '- L', '- V', '- S', '- H', '- A', '- R', '- G', '- T', '- V', '- A', '- I', '- L', '- E', '- G', '- L', '- R', '- V', '- D', '- A', '- P', '- S', '- L', '- Q', '- A', '- A', '- A', '- L', '- F', '- L', '- L', '- P', '- I', '- L', '- A', '- T', '- D', '- N', '- E', '- K', '- A', '- I', '- E', '- P', '- A', '- F', '- G', '- G', '- E', '- V', '- A', '- R', '- L', '- V', '- H', '- D', '- V', '- R', '- Q', '- L', '- L', '- R', '- I', '- G', '- A', '- I', '- A', '- G', '- M', '- A', '- S', '- P', '- A', '- E', '- A', '- G', '- V', '- T', '- R', '- K', '- N', '- E', '- A', '- E', '- A', '- R', '- R', '- A', '- Q', '- V', '- E', '- A', '- L', '- R', '- K', '- M', '- L', '- L'

In [51]:
# SAVE PROTEIN ACC FOR MAPPING

with open(aln2acc, 'wb') as handle:
    pickle.dump(g2acc, handle, protocol=pickle.HIGHEST_PROTOCOL)

# replace ambiguous bases with gaps!

In [52]:
for g in g2nucl:
    atcg = {'A', 'C', 'T', 'G'}
    ambig = set(g2nucl[g]).difference(atcg)
    
    print("before", set(g2nucl[g]))
    for b in ambig:
        g2nucl[g] = g2nucl[g].replace(b, '-')
    print("after ",set(g2nucl[g]))

before {'T', 'C', 'A', 'G'}
after  {'T', 'C', 'A', 'G'}
before {'T', 'C', 'A', 'G'}
after  {'T', 'C', 'A', 'G'}
before {'T', 'C', 'A', 'G'}
after  {'T', 'C', 'A', 'G'}
before {'T', 'C', 'A', 'G'}
after  {'T', 'C', 'A', 'G'}
before {'T', 'C', 'A', 'G'}
after  {'T', 'C', 'A', 'G'}
before {'T', 'C', 'A', 'G'}
after  {'T', 'C', 'A', 'G'}
before {'T', 'C', 'A', 'G'}
after  {'T', 'C', 'A', 'G'}
before {'T', 'C', 'A', 'G'}
after  {'T', 'C', 'A', 'G'}
before {'T', 'C', 'A', 'G'}
after  {'T', 'C', 'A', 'G'}
before {'T', 'C', 'A', 'G'}
after  {'T', 'C', 'A', 'G'}
before {'T', 'C', 'A', 'G'}
after  {'T', 'C', 'A', 'G'}
before {'T', 'C', 'A', 'G'}
after  {'T', 'C', 'A', 'G'}
before {'T', 'C', 'A', 'G'}
after  {'T', 'C', 'A', 'G'}
before {'T', 'C', 'A', 'G'}
after  {'T', 'C', 'A', 'G'}
before {'T', 'C', 'A', 'G'}
after  {'T', 'C', 'A', 'G'}
before {'T', 'C', 'A', 'G'}
after  {'T', 'C', 'A', 'G'}
before {'T', 'C', 'A', 'G'}
after  {'T', 'C', 'A', 'G'}
before {'T', 'C', 'A', 'G'}
after  {'T', 'C', 'A

In [53]:
#str(prot_db.loc["WP_097388881.1","GCF_002501565.1_ASM250156v1_protein"]['nucl'])
#for v in prot_db['GCF_001587135.1_ASM158713v1_protein'].values:
#    print(v)

# Export MSA

In [54]:
msa_fasta = open(msa_out_fasta, "w")
for g in g2nucl:
    print(g, len(g2nucl[g]))
    msa_fasta.write('>'+g+'\n') ##+g.split('_')[2][:10]+'\n')
    
    i = 0
    for gap in g2gaps[g]:
        if gap:
            msa_fasta.write('-')
        else:
            msa_fasta.write(g2nucl[g][i])
            i = i + 1
    msa_fasta.write('\n')
msa_fasta.close()

pe1 3738591
pe13 3740577
pe15 3739377
pe18 3737541
pe26 3741738
pe27 3734496
pe3 3730767
pe30 3739752
pe39 3736134
pe56 3739002
pe57 3733602
pe61 3740400
pe9 3739941
to1 3738390
to28 3738555
to36 3740154
to53 3725181
to63 3737958
to7 3738117


# Convert to Phylip

In [55]:
#SeqIO.convert(msa_out_fasta, "fasta", msa_out_phylip, "nexus")

nexus = open(msa_out_phylip, 'w')
nexus.write('#NEXUS\n')
nexus.write('begin data;\n')
nexus.write('dimensions ntax='+str(len(genomes_for_og))+' nchar='+str(len(g2gaps[g]))+' ;\n')
nexus.write('format datatype=dna ;\n')
nexus.write('matrix\n')

for record in SeqIO.parse(msa_out_fasta, "fasta"):
    nexus.write(record.description+'\t'+str(record.seq)+'\n')
    
nexus.write(';\n')
nexus.write('end;\n')

nexus.close()
#SeqIO.write(annotated_records, msa_out_phylip, "nexus")


# Export XMFA

In [56]:
msa_out_xmfa = open(msa_out_xmfa_path, 'w')

for og in ogs:
    pmsa = path_p_aln + '/' +og + '.fa'
    for record in SeqIO.parse(pmsa, "fasta"):
        #print(record.id)
        #g_id = record.id.split('_WP')[0].replace('_1_', '.1_')
        #p_id = record.id.split('_protein_')[-1]
        # prokka changes 
        ##g_id = record.id.split('_')[0]
        ##p_id = record.id.replace(record.id.split('_')[0]+'_', '')
        
        # prokka/ncbi genomes header structure 
        header = record.id.split('_')
        p_id = header[-2]+'_'+header[-1]
        g_id = strainsNames2fileNames[record.id.replace(p_id, '').replace('protein_', '')[:-1].replace('_1_', '.1_')]       
        #print(g_id, p_id)
                                           
        nns = zlib.decompress(prot_db.loc[og, g_id]['nucl']).decode() 
        #msa_out_xmfa.write('>'+record.id.split('_')[3][:10]+'\n')
        #prokka
        msa_out_xmfa.write('>'+g_id+'\n')
        
        atcg = {'A', 'T', 'G', 'C','-'}
        ambig = set(nns).difference(atcg)
        #print(ambig)
        for b in ambig:
            nns = str(nns).replace(b, '-')
        #print('aaaaaa',  nns)
        
        gaps_v = gaps2tab.loc[og, g_id]
        i = 0
        for gap in gaps_v:
            if gap:
                msa_out_xmfa.write('-')
            else:
                msa_out_xmfa.write(nns[i])
                i = i + 1
        msa_out_xmfa.write('\n')
    msa_out_xmfa.write('=' + '\n')
msa_out_xmfa.close()

# Test the position mapping
+ grab sequences from the original gb files using g2pos, g2strand and g2chr
+ load exported core OG nucleotide fasta file
+ compare the sequences 

(note: reverse compliment!)

In [57]:
if DEBUG:
    # for the compliment strand mapping
    from Bio.Seq import Seq
    Seq("A").complement()

In [58]:
if DEBUG:
    import os
    gbks = '/Users/devseeva/Desktop/work/sm_workflow/geneTrack/inputs/panX/'+ids
    ccc=0

    test_fasta = {}
    for entry in os.scandir(gbks):
        #if ccc>0:
        #    break
        ccc = ccc+1
        if (entry.path.endswith(".gbk") and entry.is_file()):
            genome = os.path.basename(entry).replace('.gbk', '').replace('_genomic', '')
            genome = strainsNames2fileNames[genome]
            test_fasta[genome] = []
            print('Genome:', genome)

            # map chr id to the sequence
            chr_seq = {}
            for record in SeqIO.parse(entry.path, "gb"):
                print('Chr',record.id, 'with', len(record.features), 'features')
                chr_seq[record.id] = record.seq

            # get corresponding mapping (from above)
            mapping_chr = g2chr[genome]
            mapping_pos = g2pos[genome]
            mapping_str = g2strand[genome]
            
            print(g2chr[genome][:10])
            print(g2pos[genome][:10])
            print(g2strand[genome][:10])
            
            print(len(mapping_chr), len(mapping_pos))
            assert(len(mapping_chr) == len(mapping_pos))

            # iterate through the original positions 
            for i in range(len(mapping_chr)):
                if mapping_str[i] == 1:
                    test_fasta[genome].append(chr_seq[mapping_chr[i]][mapping_pos[i]])
                else:
                    compl = str(Seq(chr_seq[mapping_chr[i]][mapping_pos[i]]).complement())
                    test_fasta[genome].append(compl)
            test_fasta[genome] = "".join(test_fasta[genome])
            print("Core nucl exported:", len(test_fasta[genome]))

            print()
            

Genome: to53
Chr 1 with 3246 features
Chr 2 with 1705 features
[]
[]
[]
0 0
Core nucl exported: 0

Genome: pe1
Chr NZ_WSNR01000001.1 with 6926 features
Chr NZ_WSNR01000002.1 with 3329 features
Chr NZ_WSNR01000003.1 with 129 features
Chr NZ_WSNR01000004.1 with 85 features
Chr NZ_WSNR01000005.1 with 9 features
[]
[]
[]
0 0
Core nucl exported: 0

Genome: pe15
Chr 1 with 3686 features
Chr 2 with 1613 features
Chr 5 with 163 features
Chr 6 with 22 features
[]
[]
[]
0 0
Core nucl exported: 0

Genome: pe3
Chr NZ_WSYS01000001.1 with 7134 features
Chr NZ_WSYS01000002.1 with 3309 features
Chr NZ_WSYS01000003.1 with 3 features
[]
[]
[]
0 0
Core nucl exported: 0

Genome: pe13
Chr 1 with 3720 features
Chr 2 with 1610 features
[]
[]
[]
0 0
Core nucl exported: 0

Genome: pe39
Chr NZ_WSYU01000001.1 with 7096 features
Chr NZ_WSYU01000002.1 with 3313 features
Chr NZ_WSYU01000003.1 with 87 features
Chr NZ_WSYU01000004.1 with 9 features
[]
[]
[]
0 0
Core nucl exported: 0

Genome: pe61
Chr 1 with 3651 feat

In [59]:
if DEBUG:

    for record in SeqIO.parse(msa_out_fasta, "fasta"):
        print(record.id, len(record.seq))
        print('test length', len(record.seq))
        print('Test:', test_fasta[record.id] == str(record.seq))
        print()


pe1 3761544
test length 3761544
Test: False

pe13 3761544
test length 3761544
Test: False

pe15 3761544
test length 3761544
Test: False

pe18 3761544
test length 3761544
Test: False

pe26 3761544
test length 3761544
Test: False

pe27 3761544
test length 3761544
Test: False

pe3 3761544
test length 3761544
Test: False

pe30 3761544
test length 3761544
Test: False

pe39 3761544
test length 3761544
Test: False

pe56 3761544
test length 3761544
Test: False

pe57 3761544
test length 3761544
Test: False

pe61 3761544
test length 3761544
Test: False

pe9 3761544
test length 3761544
Test: False

to1 3761544
test length 3761544
Test: False

to28 3761544
test length 3761544
Test: False

to36 3761544
test length 3761544
Test: False

to53 3761544
test length 3761544
Test: False

to63 3761544
test length 3761544
Test: False

to7 3761544
test length 3761544
Test: False



f = "/Users/devseeva/Desktop/work/pan-genome-visualization/data/ncbi-rso/metainfo.tsv"
x = pd.read_csv(f, sep='\t')  
phys = []
for a in x["accession"]:
    phys.append(a.split(".1_")[1][:10])
x["phy_id"] = phys
x.to_csv("/Users/devseeva/Desktop/work/sm_workflow/snakefiles/inputs/phy4_test_metadata/phy4_test_cleaned_data.csv", index=False)